In [47]:
% matplotlib inline

from scikits.talkbox.features import mfcc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pyaudio as pa
import os

random_state=42

音声ファイルimport

In [8]:
files = []
datapath = 'data/'

for x in os.listdir(datapath):  
    if '.wav' in x:
        files.append(x)

mfccで特徴量とラベルのセット作成

In [74]:
label_dict = {'hu':0, 'ti':1, 'dc':2}

X = []
y = []
for f in files:
    x, sample_rate = sf.read(datapath + f)
    x = np.clip(x, 1e-10, 1)
    ceps,mspec,spec = mfcc(x)
    X.append(np.mean(ceps, axis=0))
    if '-hu.' in f:
        y.append(label_dict['hu'])
    elif '-ti.' in f:
        y.append(label_dict['ti'])
    else:
        y.append(label_dict['dc'])

X = np.array(X)
y = np.array(y)

welchメソッドによるパワースペクトルで特徴量とラベルのセット作成

In [69]:
import scipy.signal

label_dict = {'hu':0, 'ti':1, 'dc':2}

X = []
y = []

fs = 44100
nperseg = 128
nfft=1024
for f in files:
    x, sample_rate = sf.read(datapath + f)
    freq, P = scipy.signal.welch(x, fs, window='hamming', nperseg=nperseg, nfft=nfft)
    X.append(P)
    if '-hu.' in f:
        y.append(label_dict['hu'])
    elif '-ti.' in f:
        y.append(label_dict['ti'])
    else:
        y.append(label_dict['dc'])

X = np.array(X)
y = np.array(y)

In [61]:
# 圧縮
from sklearn.decomposition import PCA
pca = PCA(n_components=5, random_state=random_state)
X = pca.fit_transform(X)

分類器で学習、評価

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

precision = 0
recall = 0
f_value = 0

n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    clf = LinearSVC(random_state=random_state)
    #clf = RandomForestClassifier(random_state=random_state)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_valid)
    precision += precision_score(y_valid, pred, average='macro')
    recall += recall_score(y_valid, pred, average='macro')
    f_value += f1_score(y_valid, pred, average='macro')

precision = precision / float(n_splits)
recall = recall / float(n_splits)
f_value = f_value / float(n_splits)

In [76]:
f_value

0.28553603259485616

In [77]:
y

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0, 0, 0, 2])